In [ ]:
file1 = "./data_batch_1"
def unpickle(file):
    import pickle
    with open(file, 'rb') as fo:
        dict = pickle.load(fo, encoding='bytes')
    return dict
file1 = "./data_batch_1"
files_dict1 = unpickle(file1)

In [ ]:
labels_batch1 = files_dict1[b'labels']
data_batch1 = files_dict1[b'data']
filenames_batch1 = files_dict1[b'filenames']


### Data Pre processing and functions to return BATCH_SIZE number of images

In [ ]:
import tensorflow as tf
x = tf.placeholder(tf.float32, shape=[None, 32,32,3])
y = tf.placeholder(tf.int32, shape=[None])
one_hot_y = tf.one_hot(y, 10)
def batch_norm(x, is_train=True, decay=0.99, epsilon=0.001):
    shape_x = x.get_shape().as_list()
    beta = tf.get_variable('beta', shape_x[-1], initializer=tf.constant_initializer(0.0))
    gamma = tf.get_variable('gamma', shape_x[-1], initializer=tf.constant_initializer(1.0))
    moving_mean = tf.get_variable('moving_mean', shape_x[-1],
                initializer=tf.constant_initializer(0.0), trainable=False)
    moving_var = tf.get_variable('moving_var', shape_x[-1],
               initializer=tf.constant_initializer(1.0), trainable=False)

    if is_train:
        mean, var = tf.nn.moments(x, np.arange(len(shape_x)-1), keep_dims=True)
        mean = tf.reshape(mean, [mean.shape.as_list()[-1]])
        #mean = tf.reshape(mean, [mean.get_shape().as_list()[-1]])
        var = tf.reshape(var, [var.shape.as_list()[-1]])

        update_moving_mean = tf.assign(moving_mean, moving_mean*decay + mean*(1-decay))
        update_moving_var = tf.assign(moving_var,
                            moving_var*decay + shape_x[0]/(shape_x[0]-1)*var*(1-decay))
        update_ops = [update_moving_mean, update_moving_var]

        with tf.control_dependencies(update_ops):
            return tf.nn.batch_normalization(x, mean, var, beta, gamma, epsilon)

    else:
        mean = moving_mean
        var = moving_var
        return tf.nn.batch_normalization(x, mean, var, beta, gamma, epsilon)

In [ ]:
file_test = "./test_batch"
files_dict_test = unpickle(file_test)

labels_batch_test = files_dict_test[b'labels']
data_batch_test = files_dict_test[b'data']
filenames_batch_test = files_dict_test[b'filenames']


In [ ]:
import numpy as np

def return_test_images(offset, end):
    data_test = []
    for i in range(offset, end):
        print(i)
        name = filenames_batch_test[i]

        img = np.zeros((32, 32, 3), dtype=np.uint8)
        r = (data_batch_test[i])[0:1024]

        r = np.array(r).reshape(32,32)
        g = (data_batch_test[i])[1024:2048]

        g = np.array(g).reshape(32,32)
        b = (data_batch_test[i])[2048:3072]

        b = np.array(b).reshape(32,32)
        im = np.array([r,g,b])

        rindex = 0
        gindex = 0
        bindex = 0
        for k in range(0,3):
            for i in range(0,32):
                for j in range(0,32):
                    if k == 0:
                        img[i][j][k] = r[i][j]
                        rindex = rindex + 1
                    if k == 1:
                        img[i][j][k] = g[i][j]
                        gindex = gindex + 1
                    if k == 2:
                        img[i][j][k] = b[i][j]
                        bindex = bindex + 1

        data_test.append(img)
    return data_test


### Network architecture

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten
from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm
slim = tf.contrib.slim
mu = 0
sigma = 0.1


def cifarNet(x, is_train=True):
    vs = tf.get_variable_scope()
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 32), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(32))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 2, 2, 1], padding='SAME') + conv1_b
    conv1 = batch_norm(conv1)
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')

    #Layer 2
    conv2_b = tf.Variable(tf.zeros(64))
    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 32, 64), mean = mu, stddev = sigma))
    conv2   = tf.nn.conv2d(conv1, conv2_W, strides=[1, 2, 2, 1], padding='SAME') + conv2_b
    conv2 = batch_norm(conv2)
    conv2 = tf.nn.relu(conv2)
    conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    
    #Layer 3
    conv3_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 64, 128), mean = mu, stddev = sigma))
    conv3_b = tf.Variable(tf.zeros(128))
    conv3   = tf.nn.conv2d(conv2, conv3_W, strides=[1, 2, 2, 1], padding='SAME') + conv3_b
    conv3 = batch_norm(conv3)
    conv3 = tf.nn.relu(conv3)

    conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #Layer 4
    
    conv4_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 128, 256), mean = mu, stddev = sigma))
    conv4_b = tf.Variable(tf.zeros(256))
    
    conv4   = tf.nn.conv2d(conv3, conv4_W, strides=[1, 2, 2, 1], padding='SAME') + conv4_b
    conv4 = batch_norm(conv4)
    
    conv4 = tf.nn.relu(conv4) 
    conv4 = tf.nn.max_pool(conv4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #Layer 5
    conv5_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 256, 512), mean = mu, stddev = sigma))
    conv5_b = tf.Variable(tf.zeros(512))
    
    conv5   = tf.nn.conv2d(conv4, conv5_W, strides=[1, 2, 2, 1], padding='SAME') + conv5_b
    conv5 = batch_norm(conv5)
    conv5 = tf.nn.relu(conv5)
    conv5 = tf.nn.max_pool(conv5, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #FC layer
    fc1   = flatten(conv5)
    fc1_W  = tf.Variable(tf.truncated_normal(shape=(512, 10), mean = mu, stddev = sigma))
    fc1_b  = tf.Variable(tf.zeros(10))
    logits = tf.matmul(fc1, fc1_W) + fc1_b

    return logits
    

### Reshape data to (32,32,3)

In [ ]:
import numpy as np

data_train = []

for i in range(len(filenames_batch1)): #
    name = filenames_batch1[i]

    name = str(name)
    name = name[2:-1]

    img = np.zeros((32, 32, 3), dtype=np.uint8)
    r = (data_batch1[i])[0:1024]

    r = np.array(r).reshape(32,32)
    g = (data_batch1[i])[1024:2048]

    g = np.array(g).reshape(32,32)
    b = (data_batch1[i])[2048:3072]

    b = np.array(b).reshape(32,32)
    im = np.array([r,g,b])

    rindex = 0
    gindex = 0
    bindex = 0
    for k in range(0,3):
        for i in range(0,32):
            for j in range(0,32):
                if k == 0:
                    img[i][j][k] = r[i][j]
                    rindex = rindex + 1
                if k == 1:
                    img[i][j][k] = g[i][j]
                    gindex = gindex + 1
                if k == 2:
                    img[i][j][k] = b[i][j]
                    bindex = bindex + 1
    
    data_train.append(img)

data_train = np.asarray(data_train)

In [ ]:
from sklearn.utils import shuffle

BATCH_SIZE = 1000
EPOCHS = 2000
rate_initial = 0.001
is_train = True
batch = tf.Variable(0)

logits = cifarNet(x, is_train)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
rate = tf.train.exponential_decay(rate_initial, batch*BATCH_SIZE,
                                           2000, 0.05, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

accuracy_training = []
accuracy_test = []

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = 10000
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(data_b1, labels_batch1)
        #print ("shape is " + str(X_train[i].shape))
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
        print("EPOCH {} ...".format(i+1))
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        print("Training Accuracy = {:.3f}".format(accuracy))
        print()
    test_accuracy = 0
    for offset in range(0, num_examples, BATCH_SIZE):
        end = offset + BATCH_SIZE
        batch_x, batch_y = data_test[offset:end], labels_batch_test[offset:end]
        test_accuracy = test_accuracy + sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
    
    print("test accuracy is: ", (test_accuracy/100))
        
        
    saver.save(sess, './cifarNetModel')
    print("Model saved")

### Q1 b) 

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten
from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm
slim = tf.contrib.slim
mu = 0
sigma = 0.1


def cifarNetb(x, is_train=True):
    vs = tf.get_variable_scope()
    conv1_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 3, 32), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(96))

    conv1 = tf.nn.depthwise_conv2d(x, conv1_W, strides=[1, 2, 2, 1], padding='VALID') + conv1_b
    conv1 = batch_norm(conv1)
    conv1 = tf.nn.relu(conv1)
    conv1 = tf.nn.max_pool(conv1, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    
    # 1x1 layer
    
    conv1_W_s = tf.Variable(tf.truncated_normal(shape=(1, 1, 96, 1), mean = mu, stddev = sigma))
    conv1_b_s = tf.Variable(tf.zeros(1))
    conv1_s   = tf.nn.conv2d(conv1, conv1_W_s, strides=[1, 2, 2, 1], padding='VALID') + conv1_b_s
    conv1_s = batch_norm(conv1_s)
    conv1_s = tf.nn.relu(conv1_s)
    
    #Layer 2

    conv2_b = tf.Variable(tf.zeros(64))

    conv2_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 64), mean = mu, stddev = sigma))

    conv2 = tf.nn.depthwise_conv2d(conv1_s, conv2_W, strides=[1, 2, 2, 1], padding='VALID') + conv2_b
    conv2 = batch_norm(conv2)
    conv2 = tf.nn.relu(conv2)

    
    # 1x1 layer
    
    conv2_W_s = tf.Variable(tf.truncated_normal(shape=(1, 1, 64, 1), mean = mu, stddev = sigma))
    conv2_b_s = tf.Variable(tf.zeros(1))
    conv2_s   = tf.nn.conv2d(conv2, conv2_W_s, strides=[1, 2, 2, 1], padding='VALID') + conv2_b_s
    conv2_s = batch_norm(conv2_s)
    conv2_s = tf.nn.relu(conv2_s)
    
    #Layer 3

    conv3_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 128), mean = mu, stddev = sigma))

    conv3_b = tf.Variable(tf.zeros(128))

    conv3 = tf.nn.depthwise_conv2d(conv2_s, conv3_W, strides=[1, 2, 2, 1], padding='VALID') + conv3_b
    conv3 = batch_norm(conv3)
    conv3 = tf.nn.relu(conv3)

    
    # 1x1 layer
    
    conv3_W_s = tf.Variable(tf.truncated_normal(shape=(1, 1, 128, 1), mean = mu, stddev = sigma))
    conv3_b_s = tf.Variable(tf.zeros(1))
    conv3_s   = tf.nn.conv2d(conv3, conv3_W_s, strides=[1, 2, 2, 1], padding='VALID') + conv3_b_s
    conv3_s = batch_norm(conv3_s)
    conv3_s = tf.nn.relu(conv3_s)
    
    #Layer 4
    

    conv4_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 256), mean = mu, stddev = sigma))

    conv4_b = tf.Variable(tf.zeros(256))
    

    conv4 = tf.nn.depthwise_conv2d(conv3_s, conv4_W, strides=[1, 2, 2, 1], padding='VALID') + conv4_b
    conv4 = batch_norm(conv4)
    conv4 = tf.nn.relu(conv4) 

    # 1x1 layer
    
    conv4_W_s = tf.Variable(tf.truncated_normal(shape=(1, 1, 256, 1), mean = mu, stddev = sigma))
    conv4_b_s = tf.Variable(tf.zeros(1))
    conv4_s   = tf.nn.conv2d(conv4, conv4_W_s, strides=[1, 2, 2, 1], padding='VALID') + conv4_b_s
    conv4_s = batch_norm(conv4_s)
    conv4_s = tf.nn.relu(conv4_s)
    
    
    #Layer 5
    conv5_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 1, 512), mean = mu, stddev = sigma))

    conv5_b = tf.Variable(tf.zeros(512))
    
    conv5   = tf.nn.conv2d(conv4_s, conv5_W, strides=[1, 2, 2, 1], padding='VALID') + conv5_b

    conv5 = batch_norm(conv5)
    conv5 = tf.nn.relu(conv5)
    conv5 = tf.nn.max_pool(conv5, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    #FC layer
    fc1   = flatten(conv5)
    fc1_W  = tf.Variable(tf.truncated_normal(shape=(512, 10), mean = mu, stddev = sigma))
    fc1_b  = tf.Variable(tf.zeros(10))
    logits = tf.matmul(fc1, fc1_W) + fc1_b

    return logits
    

In [ ]:
from sklearn.utils import shuffle

BATCH_SIZE = 100
EPOCHS = 2000
rate_initial = 0.001
is_train = True
batch = tf.Variable(0)

logits = cifarNetb(x, is_train)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
rate = tf.train.exponential_decay(rate_initial, batch*BATCH_SIZE,
                                           2000, 0.05, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

accuracy_training = []
accuracy_test = []

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = 10000
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(data_train, labels_batch1)
        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
        print("EPOCH {} ...".format(i+1))
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
    
    test_accuracy = 0
    for offset in range(0, num_examples, BATCH_SIZE):
        end = offset + BATCH_SIZE
        batch_x = return_test_images(offset,end)
        batch_y= labels_batch_test[offset:end]
        test_accuracy = test_accuracy + sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
    
    print("test accuracy is: ", (test_accuracy/100))
        
    saver.save(sess, './cifarNetModel')
    print("Model saved")

### Q1 c) 

In [ ]:
import tensorflow as tf
from tensorflow.contrib.layers import flatten
from tensorflow.contrib.layers.python.layers import batch_norm as batch_norm
slim = tf.contrib.slim
mu = 0
sigma = 0.1


def cifarNetc(x, is_train=True):
    vs = tf.get_variable_scope()
    conv1_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 3, 32), mean = mu, stddev = sigma))
    conv1_b = tf.Variable(tf.zeros(32))
    conv1   = tf.nn.conv2d(x, conv1_W, strides=[1, 2, 2, 1], padding='VALID') + conv1_b
    conv1 = batch_norm(conv1)
    conv1 = tf.nn.relu(conv1)

    #residual 1
    conv1_r_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 32, 32), mean = mu, stddev = sigma))
    conv1_r_b = tf.Variable(tf.zeros(32))
    conv1_r   = tf.nn.conv2d(conv1, conv1_r_W, strides=[1, 2, 2, 1], padding='VALID') + conv1_r_b
    conv1_r = batch_norm(conv1_r)
    

    #Layer 2
    conv2_b = tf.Variable(tf.zeros(64))
    conv2_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 32, 64), mean = mu, stddev = sigma))
    conv2   = tf.nn.conv2d(conv1_r, conv2_W, strides=[1, 2, 2, 1], padding='VALID') + conv2_b
    conv2 = batch_norm(conv2)
    conv2 = tf.nn.relu(conv2)
    #conv2 = tf.nn.max_pool(conv2, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #residual 2
    conv2_r_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 64, 64), mean = mu, stddev = sigma))
    conv2_r_b = tf.Variable(tf.zeros(64))
    conv2_r   = tf.nn.conv2d(conv2, conv2_r_W, strides=[1, 2, 2, 1], padding='VALID') + conv2_r_b
    conv2_r = batch_norm(conv2_r)
    
    #Layer 3
    conv3_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 64, 128), mean = mu, stddev = sigma))
    conv3_b = tf.Variable(tf.zeros(128))
    conv3   = tf.nn.conv2d(conv2_r, conv3_W, strides=[1, 2, 2, 1], padding='VALID') + conv3_b
    conv3 = batch_norm(conv3)
    conv3 = tf.nn.relu(conv3)

    #conv3 = tf.nn.max_pool(conv3, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #residual 3
    conv3_r_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 128, 128), mean = mu, stddev = sigma))
    conv3_r_b = tf.Variable(tf.zeros(128))
    conv3_r   = tf.nn.conv2d(conv3, conv3_r_W, strides=[1, 2, 2, 1], padding='VALID') + conv3_r_b
    conv3_r = batch_norm(conv3_r)
    
    #Layer 4
    
    conv4_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 128, 256), mean = mu, stddev = sigma))
    conv4_b = tf.Variable(tf.zeros(256))
    
    conv4   = tf.nn.conv2d(conv3_r, conv4_W, strides=[1, 2, 2, 1], padding='VALID') + conv4_b
    conv4 = batch_norm(conv4)
    
    conv4 = tf.nn.relu(conv4) 
    #conv4 = tf.nn.max_pool(conv4, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #residual 4
    conv4_r_W = tf.Variable(tf.truncated_normal(shape=(3, 3, 256, 256), mean = mu, stddev = sigma))
    conv4_r_b = tf.Variable(tf.zeros(256))
    conv4_r   = tf.nn.conv2d(conv4, conv4_r_W, strides=[1, 2, 2, 1], padding='VALID') + conv4_r_b
    conv4_r = batch_norm(conv4_r)
    
    
    #Layer 5
    conv5_W = tf.Variable(tf.truncated_normal(shape=(5, 5, 256, 512), mean = mu, stddev = sigma))
    conv5_b = tf.Variable(tf.zeros(512))
    
    conv5   = tf.nn.conv2d(conv4_r, conv5_W, strides=[1, 2, 2, 1], padding='VALID') + conv5_b
    conv5 = batch_norm(conv5)
    conv5 = tf.nn.relu(conv5)
    conv5 = tf.nn.max_pool(conv5, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='VALID')
    #FC layer
    fc1   = flatten(conv5)
    fc1_W  = tf.Variable(tf.truncated_normal(shape=(512, 10), mean = mu, stddev = sigma))
    fc1_b  = tf.Variable(tf.zeros(10))
    logits = tf.matmul(fc1, fc1_W) + fc1_b
    #prob = tf.nn.softmax(logits)
    return logits
    

In [ ]:
from sklearn.utils import shuffle

BATCH_SIZE = 100
EPOCHS = 2000
rate_initial = 0.001
is_train = True
batch = tf.Variable(0)

logits = cifarNetc(x, is_train)
cross_entropy = tf.nn.softmax_cross_entropy_with_logits(labels=one_hot_y, logits=logits)
loss_operation = tf.reduce_mean(cross_entropy)
rate = tf.train.exponential_decay(rate_initial, batch*BATCH_SIZE,
                                           2000, 0.05, staircase=True)
optimizer = tf.train.AdamOptimizer(learning_rate = rate)
training_operation = optimizer.minimize(loss_operation)

correct_prediction = tf.equal(tf.argmax(logits, 1), tf.argmax(one_hot_y, 1))
accuracy_operation = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))



accuracy_training = []
accuracy_test = []

saver = tf.train.Saver()

with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    num_examples = 10000
    
    print("Training...")
    print()
    for i in range(EPOCHS):
        X_train, y_train = shuffle(data_train, labels_batch1)

        for offset in range(0, num_examples, BATCH_SIZE):
            end = offset + BATCH_SIZE
            batch_x, batch_y = X_train[offset:end], y_train[offset:end]
            sess.run(training_operation, feed_dict={x: batch_x, y: batch_y})
        print("EPOCH {} ...".format(i+1))
        accuracy = sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
        print("Training Accuracy = {:.3f}".format(accuracy))
        print()

    test_accuracy = 0
    for offset in range(0, num_examples, BATCH_SIZE):
        end = offset + BATCH_SIZE
        batch_x = return_test_images(offset,end)
        batch_y= labels_batch_test[offset:end]
        test_accuracy = test_accuracy + sess.run(accuracy_operation, feed_dict={x: batch_x, y: batch_y})
    
    print("test accuracy is: ", (test_accuracy/100))
        
    saver.save(sess, './cifarNetModel')
    print("Model saved")